# **Example Road Network**

Roads are vital to the infrastructure of a community and, when blocked and/or (partially) destroyed 
cause numerous issues for various players. Therefore, it is imperative to assess the impact of loss in mobility when analyzing flood events and other hazards. 

FIAT offers a tool to compute the monetary impact of the road blockage/destruction based on the **extent of the hazard, the road type (e.g. motorway, trunk, primary roads), and the number of lanes.**

To run the FIAT model on a road network it is necessary to create the [exposure data](https://github.com/Deltares/Delft-FIAT/blob/master/docs/user_guide/data/exposure.qmd) and [vulnerability data](https://github.com/Deltares/Delft-FIAT/blob/master/docs/user_guide/data/vulnerability.qmd). So that when assessing a specific hazard event only the hazard map must be ingested to compute the impact.

To get the road network information FIAT downloads Open Street Map (OSM) data as default. Users can also ingest their own road network data.

However, we recommened OSM data as OSM offers detailed information about the road network such as the roadtype and the number of lanes, which are obligatory to calculate the flood impact. **This notebook will cover the road impact assessment based on OSM.**

>*Note: If the user wants to use their own data they musst assure to deliver the **number of lanes per segment***.

## Let's get started!

# **Step 1**: Import required packages

In [6]:
# First, all required packages must be installed.

from hydromt_fiat.fiat import FiatModel
from hydromt.log import setuplog
from pathlib import Path
import geopandas as gpd
import pandas as pd
import os 
import json
import yaml
from hydromt.config import configread
from shapely.geometry import shape, Polygon

# **Step 2:** Define the study area

FIAT requires the input of a region in form of a **shapely Polygon** to download the **OSM road dataset**. This can be done via **vector file** or **GeoJSON object** of the study area. We recommened to download the region from [Geojson.io](https://geojson.io/#map=2/0/20). 

**GeoJson.io**  
Draw a polygon on the map and create a polygon from the GeoJson Object. You can download a vector file, you need to refer to in the "region" tab in the *yaml* file using the file path. You can also copy the GeoJson code directly into the configuration file (see example in [configuration file](C:/Users/rautenba/hydromt_fiat/examples/data/road_network/configuration.yml)).

![Region_download.PNG](../docs/_static/charelston.PNG.svg)


In [15]:
# Create a dictionary of the GeoJson Object

region = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
              -79.96177042294748,
              32.79061007407526
            ],
            [
              -79.96177042294748,
              32.76748645567278
            ],
            [
              -79.91724162366485,
              32.76748645567278
            ],
            [
              -79.91724162366485,
              32.79061007407526
            ],
            [
              -79.96177042294748,
              32.79061007407526
            ]
                    ]
                ]
            }
        }
    ]
}

# Extract the coordinates
coordinates = region["features"][0]["geometry"]["coordinates"]

# Create a Shapely Polygon
polygon = Polygon(coordinates[0])  # Extract the coordinates of the exterior ring

# **Step 3:** Create the configuration file

FIAT requires a configuration file in form of a *yaml*-file or a dictionary, in which the model parameters are defined. The configuration file must define the **vulnerability** and **exposure setup**.  
The required fields are:
  
*setup_road_vulnerability:*
- **vertical_unit**:Unit of vertical inundation (feet or meter)
- **threshold_value**: Threshold inundation height at which road is fully destroyed 
- **min_hazard_value**: minimum inundation height (recommended: 0)
- **max_hazard_value**: maximum inundation height (e.g. 6 meters) 
- **step_hazard_value**: discrete levels or steps of inundation height

*setup_exposure_roads:*
- **roads_fn**: File path to road network file or OSM data (for OSM data use: "OSM")
- **road_types**: Road type keys (if OSM data use keys from from OSM e.g. "motorway", "primary")
- **road_damage**: Damages per lane and per length unit
- **unit**: Unit of road length (feet or meter)

The provided parameters need to be incorporated into the YAML structure. Please check [here](C:/Users/rautenba/hydromt_fiat/examples/data/road_network/configuration.yaml) for the setup of the *configuration.yml*. 

## **Step 3.1:** Load the configuration file

**This step is not necessary to run the model. It's just for visualization purposes.**   

The configuration file can be read in python and its information are converted into a python dictionary. </p>
<p><em>Note: Instead of loading a yaml file it is possible to create a python dictionary directly, built upon the structure seen below.</em></p>

In [16]:
# Let's read the configuration file holding the required information for the FIAT model 
with open(Path(os.path.abspath("")) / "data" / "road_network" / "configuration.yaml", 'r') as file:
    config = yaml.safe_load(file)

print(json.dumps(config, indent=4, sort_keys=False))

{
    "setup_road_vulnerability": {
        "vertical_unit": "feet",
        "threshold_value": 0.6,
        "min_hazard_value": 0,
        "max_hazard_value": 10,
        "step_hazard_value": 1
    },
    "setup_exposure_roads": {
        "roads_fn": "OSM",
        "road_types": [
            "motorway",
            "primary",
            "secondary",
            "tertiary"
        ],
        "road_damage": "default_road_max_potential_damages",
        "unit": "meter"
    }
}


# **Step 4**: <strong>Define variables for the FIAT model</strong>

To build the model some settings must be ingested as variables. So you need to set up the root path to the FIAT model, the logger settings, the region and the data catalog:
- `root`: Directory path from where the neccessary data for the FIAT model is stored. 
- `logger`: The logger variable defines the frequencies of log-entries during the initialisation of the model.
- `region`: Define the region as a variable from the config dictionary.
- `data_catalog_yml`: Define the data catalogue as a variable from the config dictionary.



In [17]:
# Set up Fiat Model
root = Path(os.path.abspath(""))  / "data" / "road_network"

# Set up the logger
logger = setuplog("hydromt_fiat", log_level=10)

# Set up datacatalog
data_catalog_yml = Path(r"C:/Users/rautenba/hydromt_fiat/examples/data/road_network/hydromt_fiat_catalog_USA.yml")

2023-10-24 09:38:12,772 - hydromt_fiat - log - INFO - HydroMT version: 0.8.0


INFO:hydromt_fiat:HydroMT version: 0.8.0


# **Step 4:** Run the FIAT model

It's time to run the road network FIAT model. The output will be stored in the root-directory. 

The output will be an **exposure folder** incl. *exposure.csv* together with vector data files, a **hazard folder** (empty) and a **vulnerability folder** incl. *damage_curve.csv*. 

In [25]:
fm = FiatModel(root=root, mode="w+", data_libs=[data_catalog_yml], logger=logger)
region = gpd.GeoDataFrame.from_features(region, crs=4326)
opt = configread(Path(os.path.abspath("")) / "data" / "road_network"  /  "configuration.yaml")
fm.build(region={"geom": region}, opt=opt)
fm.write()

2023-10-24 09:42:25,835 - hydromt_fiat - data_catalog - INFO - Parsing data catalog from C:\Users\rautenba\hydromt_fiat\examples\data\road_network\hydromt_fiat_catalog_USA.yml


INFO:hydromt_fiat:Parsing data catalog from C:\Users\rautenba\hydromt_fiat\examples\data\road_network\hydromt_fiat_catalog_USA.yml
  warnings.warn(



2023-10-24 09:42:25,838 - hydromt_fiat - model_api - INFO - Initializing fiat model from hydromt_fiat (v0.2.1.dev0).


INFO:hydromt_fiat:Initializing fiat model from hydromt_fiat (v0.2.1.dev0).


2023-10-24 09:42:25,847 - hydromt_fiat - model_api - INFO - setup_region.region: {'geom':                                             geometry
0  POLYGON ((-79.96177 32.79061, -79.96177 32.767...}


...

DEBUG:hydromt_fiat:Writing file exposure/roads.gpkg


2023-10-24 09:42:26,277 - hydromt_fiat - fiat - INFO - Writing model exposure table file to exposure/exposure.csv.


INFO:hydromt_fiat:Writing model exposure table file to exposure/exposure.csv.


2023-10-24 09:42:26,282 - hydromt_fiat - fiat - INFO - Writing model vulnerability_curves table file to vulnerability/vulnerability_curves.csv.


INFO:hydromt_fiat:Writing model vulnerability_curves table file to vulnerability/vulnerability_curves.csv.


# **Done!**
Your **FIAT model** created the exposure and vulnerability files for the road network. Using that data in combination with a hazard map, the impact for the defined hazard can be computed. 

Let's have a look at the output!

In [23]:
#Load exposure.csv into dataframe
df_exposure = pd.read_csv(Path(os.path.abspath("")) / "data" / "road_network"  /  "exposure" / "exposure.csv")

#Load exposure geopackage into GeoDataFrame
gdf_exposure =gpd.read_file(Path(os.path.abspath("")) / "data" / "road_network"  /  "exposure" /  "roads.gpkg")

#Merge dataframe with GeoDataFrame
merged_gdf = gdf_exposure.merge(df_exposure, left_on='Object ID', right_on='Object ID', how='inner')

#Display the road network
road_network = merged_gdf.explore(column = 'Max Potential Damage: Structure')
road_network

In [24]:
#Load vulnerability.csv into dataframe
df_vulnerability = pd.read_csv(Path(os.path.abspath("")) / "data" / "road_network"  /  "vulnerability" / "vulnerability_curves.csv")
df_vulnerability

,#UNIT=feet
#METHOD,mean
water depth [feet],roads
0.0,0.0
0.59,0.0
0.6,1.0
1.0,1.0
2.0,1.0
3.0,1.0
4.0,1.0
5.0,1.0
